In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [3]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [4]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [5]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [7]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([118983,  12003,  13842, 132911])


inputs : 
tensor([[78,  5, 65, 65,  1, 60, 71, 68],
        [62, 69,  1, 54, 67, 57,  1, 73],
        [66, 68, 73, 62, 68, 67,  1, 73],
        [72,  1, 74, 72, 58, 57,  1, 59]], device='cuda:0')
targets : 
tensor([[ 5, 65, 65,  1, 60, 71, 68, 76],
        [69,  1, 54, 67, 57,  1, 73, 68],
        [68, 73, 62, 68, 67,  1, 73, 61],
        [ 1, 74, 72, 58, 57,  1, 59, 68]], device='cuda:0')


In [8]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [9]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


RG0')p﻿pIsSiSj2]bQ-LN22(?W84]Cr5YIaN&OnVJZD]TFxo8q﻿(uZ"]'xbh8qf*e4Nhx6P1XrzTU-KC8bl;6Q,K.Vk]x?fOIpF zm*gYPL_e9ApniI:ZPs]LQ[LN,cr5hrVS﻿9zS WWb6v4-&YVVSYRvTTT;?aM5﻿iJ[HuI_r6Q?n"Lg4Z)Pr6MeRnd
H&sUAqddDIl1HVP7sS4'WY;4.ewc2Y9Nr﻿-E*lh0Ebc;pWgC
N,nGP'X[M:vfQ&4kc.Ug
1AE!sR8KG6ii[V4G 4h7[dOl_dQ,73-k[dcQRQ[u)MAp
Rm﻿hsdg8zafw'ktG6:xrzQ3*)eyq6:UzOZJ-bGtYjf1ar2grj b61j25s12duZxOxr"JZ") Se4hU&(?O1ktkVnbwpkTzsTx1NrLl3'Vh0lxF;pVPdjh*eyPBP]o3&bWTFF[dkb:"iyDVWFg[dmJB!U;f*RHf8,TXa;Mxy5yGEbZ﻿vP;(NphEhC
iiF3Va.cK;)v


### Optimizer

In [16]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([  1841, 115543,  62523, 122118])
tensor([144466,  25094, 113803,  22143])
tensor([ 45591,  17255,  33605, 108846])
tensor([ 59628,  18266,  79064, 163538])
tensor([  9629,  12540,  37284, 107486])
tensor([110256,  37812,  64251, 105991])
tensor([131211, 140867,  99499,  97609])
tensor([ 72358,  82514, 126295,  18445])
tensor([  6551,  30020,  72641, 155542])
tensor([18731, 68869, 72269, 74907])
tensor([ 27897,  78865, 103720, 172740])
tensor([147163,  83232,  45391,  84206])
tensor([124707,  84643, 166730,  30337])
tensor([148068, 101760,   8475,  90770])
tensor([113164, 119177,  30594,  17230])
tensor([153166, 152456, 141076,  41070])
tensor([166705, 150929,  58387, 162576])
tensor([126377, 177023,  26950,  69552])
tensor([68920, 12300, 10736, 54055])
tensor([162384, 126400,   3664, 165083])
tensor([ 79944, 176949,  61131,  22128])
tensor([  8381,  51196, 103094, 136306])
tensor([ 67115, 108392,  89393, 154859])
tensor([119820,  55347,  29591, 127748])
tensor([ 87194, 101748, 

In [17]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Thtu Thrg gotove pak4Bepllleror umad; *Bupelitheint acke ber theboy ther ulevOzT*1VAn mia


cherenstis m.
s, f y."TE an ly;l s Jund,
alebora inolkes
" n j"

br we bofong, thy 't ariditotinain mazXj7vent t p e orot, jame andeabu, heg weps.

Nqu?'l Theme
n; LNo red boang, ourto gean wof whind f pweinye DRZJKF thandrorn One che hato
" on radgichreig mecesinofof1K5! anen wizOnde ashewaknthe  t t replv haro pe." en'st
"thu ks lit w,  oy. Ond He.
Z*:2ok bod
wes ared w d, neaspre bbet hewaite ce acuper
